In [ ]:
# Cell 1: Konfigurasi
# Mengatur semua parameter dan konfigurasi yang akan digunakan di seluruh notebook

import os

# Konfigurasi data
DATA_URL = "https://github.com/evanhfw/IEEE-code/raw/refs/heads/main/data/Jerman.csv"
DATE_COLUMN = "date"
VALUE_COLUMN = "Y"
UNNECESSARY_COLUMN = "No"

# Konfigurasi fold
START_DATE_TRAIN_FOLD1 = "2015-01-01"
END_DATE_TRAIN_FOLD1 = "2017-10-31"
START_DATE_TEST_FOLD1 = "2017-11-01"
END_DATE_TEST_FOLD1 = "2017-11-30"
NUM_FOLDS = 32
FREQ = "D"

# Konfigurasi direktori output
OUTPUT_DIR = "folds_csv"
FORECAST_RESULTS_DIR = "Forecast_Results_Long"
FORECAST_PLOTS_DIR = "Forecast_Plots_Long"

# Konfigurasi Nixtla API
NIXTLA_API_KEY = "YOUR API KEY"
FORECAST_HORIZON = 30
MODEL = "timegpt-1-long-horizon"

# Instalasi dependensi
!pip install pandas numpy matplotlib nixtla -q

# Prakiraan Beban Listrik dengan TimeGPT

Notebook ini bertujuan untuk memproses data beban listrik harian dari file CSV, membuat fold untuk pelatihan dan pengujian, serta melakukan prakiraan menggunakan model TimeGPT dari Nixtla. Hasil prakiraan akan dievaluasi menggunakan metrik MAPE dan divisualisasikan.

## Langkah-langkah:
1. **Persiapan Data**: Membaca data, membersihkan kolom yang tidak diperlukan, dan memastikan tidak ada timestamp yang hilang.
2. **Pembuatan Fold**: Membagi data menjadi 32 fold untuk pelatihan dan pengujian.
3. **Prakiraan**: Menggunakan TimeGPT untuk memprediksi beban listrik pada setiap fold.
4. **Evaluasi dan Visualisasi**: Menghitung MAPE dan membuat plot perbandingan antara nilai aktual dan prediksi.

In [5]:
# Cell 2: Import Library dan Persiapan Data
# Mengimpor library yang diperlukan dan memproses data awal

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from nixtla import NixtlaClient

# Membaca data dari URL
print("Membaca data dari URL...")
df = pd.read_csv(DATA_URL)

# Menghapus kolom yang tidak diperlukan
if UNNECESSARY_COLUMN in df.columns:
    df = df.drop(columns=[UNNECESSARY_COLUMN])
    print(f"Kolom '{UNNECESSARY_COLUMN}' telah dihapus.")

# Mengonversi kolom tanggal menjadi datetime dan mengatur sebagai indeks
df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
df = df.set_index(DATE_COLUMN)
print(f"Data telah diatur dengan indeks '{DATE_COLUMN}'.")

# Memeriksa timestamp yang hilang
full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=FREQ)
missing_timestamps = full_range.difference(df.index)

if not missing_timestamps.empty:
    print(f"Terdapat {len(missing_timestamps)} timestamp yang hilang.")
    print(f"Contoh timestamp yang hilang (10 pertama):\n{missing_timestamps[:10]}")
    df = df.reindex(full_range).ffill()
    print("Data yang hilang telah diisi menggunakan metode forward fill.")
else:
    print("Tidak ada timestamp yang hilang.")

Membaca data dari URL...
Data telah diatur dengan indeks 'date'.
Tidak ada timestamp yang hilang.


In [6]:
# Cell 3: Pembuatan Fold untuk Pelatihan dan Pengujian
# Membagi data menjadi fold pelatihan dan pengujian berdasarkan konfigurasi

# Inisialisasi list untuk menyimpan fold
training_folds = []
testing_folds = []

# Membuat direktori output jika belum ada
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Mengonversi tanggal konfigurasi ke timestamp
start_train = pd.Timestamp(START_DATE_TRAIN_FOLD1)
end_train = pd.Timestamp(END_DATE_TRAIN_FOLD1)
start_test = pd.Timestamp(START_DATE_TEST_FOLD1)
end_test = pd.Timestamp(END_DATE_TEST_FOLD1)

print(f"Membuat {NUM_FOLDS} fold untuk pelatihan dan pengujian...")

for i in range(NUM_FOLDS):
    # Menghitung tanggal untuk fold saat ini
    current_start_train = start_train + timedelta(days=i)
    current_end_train = end_train + timedelta(days=i)
    current_start_test = start_test + timedelta(days=i)
    current_end_test = end_test + timedelta(days=i)

    # Memilih data untuk fold pelatihan dan pengujian
    train_fold = df.loc[current_start_train:current_end_train]
    test_fold = df.loc[current_start_test:current_end_test]

    # Menyimpan fold ke list
    training_folds.append(train_fold)
    testing_folds.append(test_fold)

    # Mengembalikan indeks sebagai kolom untuk penyimpanan
    train_fold = train_fold.reset_index()
    test_fold = test_fold.reset_index()

    # Menyimpan fold ke file CSV
    train_filename = os.path.join(OUTPUT_DIR, f"training_fold_{i+1}.csv")
    test_filename = os.path.join(OUTPUT_DIR, f"testing_fold_{i+1}.csv")
    train_fold.to_csv(train_filename, index=False)
    test_fold.to_csv(test_filename, index=False)

    # Menampilkan informasi fold
    print(f"Fold {i+1}:")
    print(f"  Pelatihan: {current_start_train} sampai {current_end_train} ({len(train_fold)} data)")
    print(f"  Pengujian: {current_start_test} sampai {current_end_test} ({len(test_fold)} data)")
    print(f"  File pelatihan: {train_filename}")
    print(f"  File pengujian: {test_filename}\n")

Membuat 32 fold untuk pelatihan dan pengujian...
Fold 1:
  Pelatihan: 2015-01-01 00:00:00 sampai 2017-10-31 00:00:00 (1035 data)
  Pengujian: 2017-11-01 00:00:00 sampai 2017-11-30 00:00:00 (30 data)
  File pelatihan: folds_csv/training_fold_1.csv
  File pengujian: folds_csv/testing_fold_1.csv

Fold 2:
  Pelatihan: 2015-01-02 00:00:00 sampai 2017-11-01 00:00:00 (1035 data)
  Pengujian: 2017-11-02 00:00:00 sampai 2017-12-01 00:00:00 (30 data)
  File pelatihan: folds_csv/training_fold_2.csv
  File pengujian: folds_csv/testing_fold_2.csv

Fold 3:
  Pelatihan: 2015-01-03 00:00:00 sampai 2017-11-02 00:00:00 (1035 data)
  Pengujian: 2017-11-03 00:00:00 sampai 2017-12-02 00:00:00 (30 data)
  File pelatihan: folds_csv/training_fold_3.csv
  File pengujian: folds_csv/testing_fold_3.csv

Fold 4:
  Pelatihan: 2015-01-04 00:00:00 sampai 2017-11-03 00:00:00 (1035 data)
  Pengujian: 2017-11-04 00:00:00 sampai 2017-12-03 00:00:00 (30 data)
  File pelatihan: folds_csv/training_fold_4.csv
  File pengujia

In [7]:
# Cell 4: Inspeksi Fold
# Menampilkan informasi singkat tentang fold pertama dan terakhir

# Mengakses fold pertama dan terakhir
fold1_train = training_folds[0].reset_index()
fold1_test = testing_folds[0].reset_index()
fold_last_train = training_folds[NUM_FOLDS-1].reset_index()
fold_last_test = testing_folds[NUM_FOLDS-1].reset_index()

# Menampilkan informasi
print("Informasi Fold 1 - Pelatihan:")
print(fold1_train.head())
print(f"\nTail Fold 1 - Pelatihan:\n{fold1_train.tail()}")

print("\nInformasi Fold 1 - Pengujian:")
print(fold1_test.head())
print(f"\nTail Fold 1 - Pengujian:\n{fold1_test.tail()}")

print(f"\nInformasi Fold {NUM_FOLDS} - Pelatihan:")
print(fold_last_train.head())
print(f"\nTail Fold {NUM_FOLDS} - Pelatihan:\n{fold_last_train.tail()}")

print(f"\nInformasi Fold {NUM_FOLDS} - Pengujian:")
print(fold_last_test.head())
print(f"\nTail Fold {NUM_FOLDS} - Pengujian:\n{fold_last_test.tail()}")

Informasi Fold 1 - Pelatihan:
        date         y
0 2015-01-01  1111.336
1 2015-01-02  1300.884
2 2015-01-03  1265.271
3 2015-01-04  1198.854
4 2015-01-05  1449.861

Tail Fold 1 - Pelatihan:
           date           y
1030 2017-10-27  1480.50323
1031 2017-10-28  1284.26158
1032 2017-10-29  1176.91823
1033 2017-10-30  1345.87389
1034 2017-10-31  1204.08577

Informasi Fold 1 - Pengujian:
        date           y
0 2017-11-01  1309.18478
1 2017-11-02  1474.19484
2 2017-11-03  1471.18241
3 2017-11-04  1284.77485
4 2017-11-05  1193.85495

Tail Fold 1 - Pengujian:
         date           y
25 2017-11-26  1276.09818
26 2017-11-27  1602.17797
27 2017-11-28  1615.38061
28 2017-11-29  1603.98435
29 2017-11-30  1617.03309

Informasi Fold 32 - Pelatihan:
        date         y
0 2015-02-01  1273.828
1 2015-02-02  1608.680
2 2015-02-03  1639.260
3 2015-02-04  1651.314
4 2015-02-05  1647.357

Tail Fold 32 - Pelatihan:
           date           y
1030 2017-11-27  1602.17797
1031 2017-11-28  1615.

In [8]:
# Cell 5: Prakiraan dengan TimeGPT dan Evaluasi
# Melakukan prakiraan untuk setiap fold dan menghitung MAPE

# Inisialisasi Nixtla Client
nixtla_client = NixtlaClient(api_key=NIXTLA_API_KEY)

# Memvalidasi API key
if nixtla_client.validate_api_key():
    print("API Key valid.")
else:
    raise ValueError("API Key tidak valid.")

# Membuat direktori untuk hasil dan plot
os.makedirs(FORECAST_RESULTS_DIR, exist_ok=True)
os.makedirs(FORECAST_PLOTS_DIR, exist_ok=True)

# Inisialisasi list untuk menyimpan hasil MAPE
mape_results = []

print("Memulai prakiraan untuk semua fold...")

for fold in range(1, NUM_FOLDS + 1):
    # Memuat data pelatihan dan pengujian
    train_file = os.path.join(OUTPUT_DIR, f"training_fold_{fold}.csv")
    test_file = os.path.join(OUTPUT_DIR, f"testing_fold_{fold}.csv")
    
    df = pd.read_csv(train_file)
    df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
    df['unique_id'] = 'ts_1'
    df.rename(columns={DATE_COLUMN: 'ds', VALUE_COLUMN: 'y'}, inplace=True)
    
    test = pd.read_csv(test_file)
    test[DATE_COLUMN] = pd.to_datetime(test[DATE_COLUMN])
    test['unique_id'] = 'ts_1'
    test.rename(columns={DATE_COLUMN: 'ds', VALUE_COLUMN: 'y'}, inplace=True)

    # Melakukan prakiraan
    print(f"Melakukan prakiraan untuk fold {fold}...")
    timegpt_fcst_df = nixtla_client.forecast(
        df=df,
        h=FORECAST_HORIZON,
        freq=FREQ,
        time_col='ds',
        target_col='y',
        model=MODEL
    )

    # Menyimpan hasil prakiraan
    forecast_file = os.path.join(FORECAST_RESULTS_DIR, f"TimeGPT_Fold{fold}.csv")
    timegpt_fcst_df.to_csv(forecast_file, index=False)
    print(f"Hasil prakiraan fold {fold} disimpan di: {forecast_file}")

    # Menggabungkan data pengujian dengan prakiraan untuk evaluasi
    merged = pd.merge(test, timegpt_fcst_df, on=['unique_id', 'ds'], how='inner')
    merged['error'] = abs(merged['y'] - merged['TimeGPT'])
    merged['percentage_error'] = merged['error'] / merged['y'] * 100
    mape = merged['percentage_error'].mean()
    mape_results.append({'Fold': fold, 'MAPE': mape})
    print(f"MAPE untuk fold {fold}: {mape:.2f}%")

    # Membuat plot aktual vs prediksi
    plt.figure(figsize=(10, 6))
    plt.plot(merged['ds'], merged['y'], label='Aktual', marker='o')
    plt.plot(merged['ds'], merged['TimeGPT'], label='Prediksi', linestyle='--')
    plt.xlabel('Tanggal')
    plt.ylabel('Beban Listrik (MWh)')
    plt.title(f'Aktual vs Prediksi Beban Listrik - Fold {fold}')
    plt.legend()
    plt.grid(True)
    plot_file = os.path.join(FORECAST_PLOTS_DIR, f"Fold_{fold}_Actual_vs_Predicted.png")
    plt.savefig(plot_file)
    plt.close()
    print(f"Plot fold {fold} disimpan di: {plot_file}")

# Menyimpan hasil MAPE
mape_df = pd.DataFrame(mape_results)
mape_file = os.path.join(FORECAST_RESULTS_DIR, "MAPE_Summary.csv")
mape_df.to_csv(mape_file, index=False)
print(f"Ringkasan MAPE disimpan di: {mape_file}")

# Menampilkan ringkasan MAPE
print("\nRingkasan MAPE untuk semua fold:")
print(mape_df)

API Key valid.
Memulai prakiraan untuk semua fold...
Melakukan prakiraan untuk fold 1...
Hasil prakiraan fold 1 disimpan di: Forecast_Results_Long/TimeGPT_Fold1.csv
MAPE untuk fold 1: 8.74%
Plot fold 1 disimpan di: Forecast_Plots_Long/Fold_1_Actual_vs_Predicted.png
Melakukan prakiraan untuk fold 2...
Hasil prakiraan fold 2 disimpan di: Forecast_Results_Long/TimeGPT_Fold2.csv
MAPE untuk fold 2: 9.31%
Plot fold 2 disimpan di: Forecast_Plots_Long/Fold_2_Actual_vs_Predicted.png
Melakukan prakiraan untuk fold 3...
Hasil prakiraan fold 3 disimpan di: Forecast_Results_Long/TimeGPT_Fold3.csv
MAPE untuk fold 3: 8.18%
Plot fold 3 disimpan di: Forecast_Plots_Long/Fold_3_Actual_vs_Predicted.png
Melakukan prakiraan untuk fold 4...
Hasil prakiraan fold 4 disimpan di: Forecast_Results_Long/TimeGPT_Fold4.csv
MAPE untuk fold 4: 8.38%
Plot fold 4 disimpan di: Forecast_Plots_Long/Fold_4_Actual_vs_Predicted.png
Melakukan prakiraan untuk fold 5...
Hasil prakiraan fold 5 disimpan di: Forecast_Results_Long/T

ApiError: status_code: 429, body: {'detail': 'You have reached your request limit per month, email ops@nixtla.io to increase your usage limit or wait until next month'}